In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import mlflow
# Global settings
n_jobs = -1 # This parameter conrols the parallel processing. -1 means using all processors.
random_state = 40

In [ ]:

# URL del archivo CSV en el repositorio de GitHub
url1 = 'https://raw.githubusercontent.com/jsalcedo14/DSA_project/master/dataTrain_carListings.csv'
url2 = 'https://raw.githubusercontent.com/jsalcedo14/DSA_project/master/dataTest_carListings.csv'
# Lee el archivo CSV en un DataFrame de pandas
data = pd.read_csv(url1)
test = pd.read_csv(url2)

In [ ]:
# Codificación de las variables categoricas
cat = ['State','Make','Model']
dic = {'State':{},'Make':{},'Model':{}}

for i in cat:
    idx, codex = pd.factorize(data[i])
    data[i] = idx
    # Diccionario de referencia
    dic[i].update({code: i for i, code in enumerate(codex)})
    # Codificación de variables en testing con base en el diccionario
    test[i] = test[i].apply(lambda code: dic[i][code])

In [ ]:
print('Cantidad Datos:',len(data))

# Control de Outliers
Q1 = data['Price'].quantile(0.25)
print('Primer Cuartil:', Q1)

Q3 = data['Price'].quantile(0.75)
print('Tercer Cuartil:', Q3)

IQR = Q3 - Q1
print('Rango Intercuartil:', IQR)

Mediana = data['Price'].median()
print('Mediana:', Mediana)

Valor_Minimo = data['Price'].min()
print('Valor Mínimo:', Valor_Minimo)

Valor_Maximo = data['Price'].max()
print('Valor Máximo:', Valor_Maximo)

BI_Calculado = (Q1 - 1.5 * IQR)
print('BI_Calculado: \n', BI_Calculado)

BS_Calculado = (Q3 + 1.5 * IQR)
print('BS_Calculado: \n', BS_Calculado)

Cantidad Datos: 400000
Primer Cuartil: 13499.0
Tercer Cuartil: 26999.0
Rango Intercuartil: 13500.0
Mediana: 18450.0
Valor Mínimo: 5001
Valor Máximo: 79999
BI_Calculado: 
 -6751.0
BS_Calculado: 
 47249.0


In [ ]:
ubicacion_outliers = (data['Price'] < BI_Calculado) | (data['Price'] > BS_Calculado)
outliers = data[ubicacion_outliers]
print('\n Lista de Outliers \n', outliers)

ubicacion_sin_out = (data['Price'] >= BI_Calculado) & (data['Price'] <= BS_Calculado)
data = data[ubicacion_sin_out]


 Lista de Outliers 
         Price  Year  Mileage  State  Make  Model
90      61700  2017      355      5     1     68
122     49977  2015    25740      7     1     86
128     47550  2011    47383     22    22     91
168     51400  2015    44905      2    10    105
187     73216  2016     4912      2     1    117
...       ...   ...      ...    ...   ...    ...
399900  59980  2017    10989      2     1     45
399931  67365  2017     4519      0     9    144
399956  63900  2016    21501     32    22     91
399957  52998  2017     4694     25     2    230
399992  50990  2017    23896      2    27    162

[10880 rows x 6 columns]


In [ ]:
# Selección de variables numéricas (X)

X = data.drop(['Price'], axis=1)

X['YxM'] = X['Year'] * X['Mileage']
test['YxM'] = test['Year'] * test['Mileage']

y = data['Price']

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)

In [ ]:

# defina el servidor para llevar el registro de modelos y artefactos
# mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("/GradientB")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas.
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo.
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # defina los parámetros del modelo
    n_estimators = 248
    learning_rate = 0.2
    max_depth = 10
    max_features = 0.7
    subsample = 1

    # Cree el modelo con los parámetros definidos y entrénelo
    gb = GradientBoostingRegressor(n_estimators=n_estimators,
                                   learning_rate=learning_rate,
                                   max_depth=max_depth,
                                   max_features=max_features,
                                   subsample=subsample)
    gb.fit(X_train, y_train)

    # Realice predicciones de prueba
    predictions = gb.predict(X_test)

    # Calcule y registre las métricas
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100.0

    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)
    mlflow.log_param("subsample", subsample)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mape", mape)

    # Registre el modelo
    mlflow.sklearn.log_model(gb, "gradient-boosting-model")

    print("MSE:", mse)
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("MAPE:", mape)


2023/11/13 00:38:48 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.8.0/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

MSE: 8847315.897443125
RMSE: 2974.443796316065
MAE: 2052.219503004961
MAPE: 10.971310457195347
